In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../scraper/src/data/ratebeer_raw.csv')

c:\users\pinaa\documents\git-repo\tvtime-dataviz\jupy_venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ratebeer_df = df.copy()

In [4]:
# Set review_id as index.
ratebeer_df.columns = ['review_id'] + list(ratebeer_df.columns)[1:]
ratebeer_df.set_index('review_id');

In [5]:
# Transform features to numeric.
import re
def str_to_int(x):
    if isinstance(x, str):
        return int(re.findall(r"\d+", x)[-1])
    return int(x)

def str_to_float(x):
    if isinstance(x, str):
        try:
            return float(re.findall(r"[-+]?\d*\.\d+|\d+", x)[-1]) if x != '-' else None
        except IndexError:
            print(x)
    return float(x)

def get_review(x):
    return float(re.findall(r"\d+", x)[0])

def scale(series):
    max_value = series.max()
    return series.map(lambda x: x / max_value * 10)

ratebeer_df.dropna(subset=['beer_id'], inplace=True)
ratebeer_df['beer_id'] = ratebeer_df['beer_id'].map(str_to_int)
ratebeer_df['brewer_id'] = ratebeer_df['brewer_id'].map(str_to_int)
ratebeer_df['beer_ABV'] = ratebeer_df['beer_ABV'].map(str_to_float)
ratebeer_df['review_appearance'] = ratebeer_df['review_appearance'].map(get_review)
ratebeer_df['review_aroma'] = ratebeer_df['review_aroma'].map(get_review)
ratebeer_df['review_palate'] = ratebeer_df['review_palate'].map(get_review)
ratebeer_df['review_taste'] = ratebeer_df['review_taste'].map(get_review)
ratebeer_df['review_overall'] = ratebeer_df['review_overall'].map(get_review)
# Scale values.
ratebeer_df['review_appearance'] = scale(ratebeer_df['review_appearance'])
ratebeer_df['review_aroma'] = scale(ratebeer_df['review_aroma'])
ratebeer_df['review_palate'] = scale(ratebeer_df['review_palate'])
ratebeer_df['review_taste'] = scale(ratebeer_df['review_taste'])
ratebeer_df['review_overall'] = scale(ratebeer_df['review_overall'])

In [6]:
ratebeer_df.describe().loc[:,'beer_ABV':'review_overall'].apply(lambda s: s.apply(lambda x: format(x, '.2f')))

,beer_ABV,review_appearance,review_aroma,review_palate,review_taste,review_overall
count,2785526.00,2924163.00,2924163.00,2924163.00,2924163.00,2924163.00
mean,6.64,6.86,6.35,6.51,6.45,6.60
std,2.31,1.63,1.64,1.66,1.62,1.68
min,0.01,2.00,1.00,2.00,1.00,0.50
25%,5.00,6.00,6.00,6.00,6.00,6.00
50%,6.00,6.00,7.00,6.00,7.00,7.00
75%,8.00,8.00,7.00,8.00,8.00,7.50
max,57.70,10.00,10.00,10.00,10.00,10.00


In [7]:
ratebeer_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2924163 entries, 0 to 2924162
Data columns (total 14 columns):
 #   Column             Dtype  
---  ------             -----  
 0   review_id          int64  
 1   beer_name          object 
 2   beer_id            int64  
 3   brewer_id          int64  
 4   beer_ABV           float64
 5   beer_style         object 
 6   review_appearance  float64
 7   review_aroma       float64
 8   review_palate      float64
 9   review_taste       float64
 10  review_overall     float64
 11  review_time        float64
 12  review_user        object 
 13  review_text        object 
dtypes: float64(7), int64(3), object(4)
memory usage: 334.6+ MB


In [8]:
grouped_beers = ratebeer_df.groupby(['beer_id'])

In [9]:
distinct_beers = {
    'beer_id': grouped_beers.count().index,
    'beer_name': grouped_beers.first()['beer_name'],
    'n_reviews': grouped_beers.count()['review_id'],
    'brewer_id': grouped_beers.first()['brewer_id'],
    'beer_abv': grouped_beers.mean()['beer_ABV'],
    'beer_style': grouped_beers.first()['beer_style'],
    'appearance_mean': grouped_beers.mean()['review_appearance'],
    'appearance_std': grouped_beers.std()['review_appearance'],
    'aroma_mean': grouped_beers.mean()['review_aroma'],
    'aroma_std': grouped_beers.std()['review_aroma'],
    'palate_mean': grouped_beers.mean()['review_palate'],
    'palate_std': grouped_beers.std()['review_palate'],
    'taste_mean': grouped_beers.mean()['review_taste'],
    'taste_std': grouped_beers.std()['review_taste'],
    'overall_mean': grouped_beers.mean()['review_overall'],
    'overall_std': grouped_beers.std()['review_overall'],
}

In [10]:
distinct_beers_df = pd.DataFrame(distinct_beers)
distinct_beers_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110364 entries, 1 to 163404
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   beer_id          110364 non-null  int64  
 1   beer_name        110364 non-null  object 
 2   n_reviews        110364 non-null  int64  
 3   brewer_id        110364 non-null  int64  
 4   beer_abv         87244 non-null   float64
 5   beer_style       110364 non-null  object 
 6   appearance_mean  110364 non-null  float64
 7   appearance_std   85798 non-null   float64
 8   aroma_mean       110364 non-null  float64
 9   aroma_std        85798 non-null   float64
 10  palate_mean      110364 non-null  float64
 11  palate_std       85798 non-null   float64
 12  taste_mean       110364 non-null  float64
 13  taste_std        85798 non-null   float64
 14  overall_mean     110364 non-null  float64
 15  overall_std      85798 non-null   float64
dtypes: float64(11), int64(3), object(2)
me

In [11]:
distinct_beers_df.describe().apply(lambda s: s.apply(lambda x: format(x, '.2f')))

,beer_id,n_reviews,brewer_id,beer_abv,appearance_mean,appearance_std,aroma_mean,aroma_std,palate_mean,palate_std,taste_mean,taste_std,overall_mean,overall_std
count,110364.00,110364.00,110364.00,87244.00,110364.00,85798.00,110364.00,85798.00,110364.00,85798.00,110364.00,85798.00,110364.00,85798.00
mean,79995.56,26.50,4138.20,5.91,6.60,1.10,6.13,0.94,6.27,1.12,6.22,0.97,6.31,0.94
std,46925.16,111.88,3622.50,2.11,1.10,0.59,1.15,0.50,1.13,0.60,1.17,0.52,1.23,0.52
min,1.00,1.00,1.00,0.01,2.00,0.00,1.00,0.00,2.00,0.00,1.00,0.00,0.50,0.00
25%,39060.75,2.00,1065.75,4.60,6.00,0.97,5.57,0.70,5.96,0.97,5.67,0.71,5.75,0.62
50%,79603.50,4.00,3074.00,5.30,6.57,1.15,6.20,0.92,6.00,1.15,6.33,0.96,6.50,0.89
75%,120307.25,13.00,6299.75,6.80,7.33,1.41,7.00,1.20,7.00,1.41,7.00,1.24,7.06,1.20
max,163404.00,3696.00,13946.00,57.70,10.00,5.66,10.00,5.66,10.00,5.66,10.00,5.66,10.00,6.36


In [13]:
distinct_beers_df.to_csv('../scraper/src/data/ratebeer.csv', index=False)